In [1]:
import numpy as np
import pickle as pkl
import plotly.graph_objects as go
import pickle as pkl
import torch
import numpy as np
import plotly.graph_objects as go
import plotly.express as px
from scipy.stats import entropy
import sklearn.metrics as metrics
import pickle as pkl
import pandas as pd
import torch
from sklearn.model_selection import train_test_split

from denoisers.ConditionalUnetDenoiser import ConditionalUnetDenoiser
from denoisers.ConditionalUnetMatrixDenoiser import ConditionalUnetMatrixDenoiser
from utils.graph_utils import get_process_model_reachability_graph_transition_matrix, \
    get_process_model_petri_net_transition_matrix, get_process_model_reachability_graph_transition_multimatrix
from utils.pm_utils import discover_dk_process, remove_duplicates_dataset, pad_to_multiple_of_n
from utils.Config import Config
from dataset.dataset import SaladsDataset
from ddpm.ddpm_multinomial import Diffusion
import os
import json
from torch.utils.data import DataLoader
from tqdm.notebook import tqdm
from utils.pm_utils import conformance_measure
from scipy.stats import wasserstein_distance
from sklearn.metrics import roc_auc_score, accuracy_score, precision_score, recall_score, f1_score
import sklearn.metrics as metrics
from src.denoisers.ConditionalUnetGraphDenoiser import ConditionalUnetGraphDenoiser
from src.utils.graph_utils import prepare_process_model_for_gnn, \
    get_process_model_reachability_graph_transition_multimatrix

In [15]:
def get_beta_params(low, high, middle):
    """
    Determines Beta parameters such that when sampling X ~ Beta(a, b)
    and scaling alpha = low + (high - low) * X, we have P(alpha < 0.5) ≈ middle.
    Here we fix a = 1 and solve for b.

    Parameters:
      low    : lower bound of the target range.
      high   : upper bound of the target range.
      middle : desired probability that alpha is less than 0.5.

    Returns:
      (a, b) : tuple of Beta parameters.
    """
    a = 1.0
    q = (0.5 - low) / (high - low)
    b = np.log(1 - middle) / np.log(1 - q)
    return a, b


def sample_noise_mask(T,
                      p_clean_to_noisy=0.01,
                      p_noisy_to_clean=0.1):
    """
    Returns a boolean mask of length T.  True = noisy state, False = clean.
    The expected run‐length in noisy state is ~1/p_noisy_to_clean,
    and the long‐run fraction of time spent in noisy state is
        π_noisy = p_clean_to_noisy / (p_clean_to_noisy + p_noisy_to_clean)
    """
    state = False  # start clean
    mask = np.zeros(T, dtype=bool)
    for t in range(T):
        if not state:
            # clean → noisy?
            if np.random.rand() < p_clean_to_noisy:
                state = True
        else:
            # noisy → clean?
            if np.random.rand() < p_noisy_to_clean:
                state = False
        mask[t] = state
    return mask


def generate_locked_bursty_trace(target,
                                 low=0.05, high=1.0, middle=0.8,
                                 noise_level_clean=0.0,
                                 noise_level_noisy=0.6,
                                 p_c2n=0.005, p_n2c=0.02):
    """
    Same as before, but in each noisy segment we pick a single wrong class j
    and blend (1-noise_level)*one_hot(j) + noise_level*Dirichlet_noise,
    so argmax stays j throughout the burst.
    """
    T, K = target.shape
    # 1) sample the clean/noisy mask
    noisy_mask = sample_noise_mask(T, p_c2n, p_n2c)

    # 2) precompute your Beta->Dirichlet concentration if you like
    a, b = get_beta_params(low, high, middle)

    noisy_trace = []
    t = 0
    while t < T:
        if not noisy_mask[t]:
            # clean: just keep original p exactly
            p = target[t]
            lvl = noise_level_clean
            # fresh Dirichlet noise around uniform
            alpha_vec = np.ones(K) * np.random.beta(a, b)
            noise = np.random.dirichlet(alpha_vec)
            perturbed = (1 - lvl) * p + lvl * noise
            if np.any(np.isnan(perturbed)):
                    perturbed = p
            noisy_trace.append(perturbed / perturbed.sum())
            t += 1
        else:
            # find the length of this noisy run
            start = t
            while t < T and noisy_mask[t]:
                t += 1
            end = t

            # pick one wrong class j for the _whole_ run
            true_label = target[start].argmax()
            # choose j != true_label
            candidates = list(range(K))
            candidates.remove(true_label)
            j = np.random.choice(candidates)

            # now fill in each time‐step in [start, end)
            for _ in range(start, end):
                # draw fresh Dirichlet around uniform base (or you can weight it)
                alpha_vec = np.ones(K) * np.random.beta(a, b)
                noise = np.random.dirichlet(alpha_vec)

                # mix with constant one_hot(j)
                lvl = noise_level_noisy
                one_hot_j = np.eye(K)[j]
                perturbed = (1 - lvl) * one_hot_j + lvl * noise
                # it already sums to 1, so no need to renormalize
                if np.any(np.isnan(perturbed)):
                    perturbed = one_hot_j
                noisy_trace.append(perturbed)

    return np.stack(noisy_trace)


def visualize_traces(x, y, z, names=None):
    if names is None:
        names = ['Original', 'Argmax', 'Reconstructed']
    fig = go.Figure()
    fig.add_trace(go.Scatter(x=list(range(len(x))), y=x, mode='lines', name=names[0],
                             line=dict(color='blue', dash='solid')))
    fig.add_trace(go.Scatter(x=list(range(len(y))), y=y, mode='lines', name=names[1],
                             line=dict(color='red', dash='dash')))
    fig.add_trace(go.Scatter(x=list(range(len(z))), y=z, mode='lines', name=names[2],
                             line=dict(color='green', dash='dot')))
    return fig


def average_entropy_data(data):
    return np.mean([np.mean(entropy(x, axis=1)) for x in data])


def load_experiment_config(target_dir):
    config_path = os.path.join(target_dir, "cfg.json")
    if os.path.exists(config_path):
        with open(config_path, "r") as f:
            return Config(**json.load(f))
    else:
        st.warning("Configuration file not found.")
        return None


def load_experiment_data_and_model(target_dir, cfg):
    with open(cfg.data_path, "rb") as f:
        base_dataset = pkl.load(f)
    dataset = SaladsDataset(base_dataset['target'], base_dataset['stochastic'])
    train_dataset, test_dataset = train_test_split(dataset, train_size=cfg.train_percent, shuffle=True,
                                                   random_state=cfg.seed)
    rg_transition_matrix = torch.randn((cfg.num_classes, 2, 2)).to(cfg.device)
    dk_process_model, dk_init_marking, dk_final_marking = discover_dk_process(train_dataset, cfg,
                                                                              preprocess=remove_duplicates_dataset)
    if cfg.enable_matrix:
        if cfg.matrix_type == "pm":
            rg_nx, rg_transition_matrix = get_process_model_petri_net_transition_matrix(dk_process_model,
                                                                                        dk_init_marking,
                                                                                        dk_final_marking)
            rg_transition_matrix = torch.tensor(rg_transition_matrix, device=cfg.device).unsqueeze(0).float()
        elif cfg.matrix_type == "rg":
            rg_nx, rg_transition_matrix = get_process_model_reachability_graph_transition_multimatrix(dk_process_model,
                                                                                                      dk_init_marking)
            rg_transition_matrix = torch.tensor(rg_transition_matrix, device=cfg.device).float()
        rg_transition_matrix = pad_to_multiple_of_n(rg_transition_matrix)
    if cfg.enable_gnn:
        pm_nx_data = prepare_process_model_for_gnn(dk_process_model, dk_init_marking, dk_final_marking,
                                                   cfg).to(cfg.device)

    diffuser = Diffusion(noise_steps=cfg.num_timesteps, device=cfg.device)
    if cfg.enable_matrix:
        denoiser = ConditionalUnetMatrixDenoiser(in_ch=cfg.num_classes, out_ch=cfg.num_classes,
                                                 max_input_dim=dataset.sequence_length,
                                                 transition_dim=rg_transition_matrix.shape[-1],
                                                 gamma=cfg.gamma,
                                                 matrix_out_channels=rg_transition_matrix.shape[0],
                                                 device=cfg.device).to(cfg.device).float()
    elif cfg.enable_gnn:
        denoiser = ConditionalUnetGraphDenoiser(in_ch=cfg.num_classes, out_ch=cfg.num_classes,
                                                max_input_dim=dataset.sequence_length,
                                                num_nodes=pm_nx_data.num_nodes,
                                                graph_data=pm_nx_data,
                                                embedding_dim=128, hidden_dim=128, pooling=cfg.gnn_pooling).to(cfg.device).float()
    else:
        denoiser = ConditionalUnetDenoiser(in_ch=cfg.num_classes, out_ch=cfg.num_classes,
                                           max_input_dim=dataset.sequence_length,
                                           device=cfg.device).to(cfg.device).float()
    ckpt_path = os.path.join(target_dir, "best.ckpt")
    denoiser.load_state_dict(torch.load(ckpt_path, map_location=cfg.device)['model_state'])
    final_res_path = os.path.join(target_dir, "final_results.json")
    if os.path.exists(final_res_path):
        with open(final_res_path, "r") as f:
            final_res = json.load(f)

    return (train_dataset, test_dataset, dk_process_model, dk_init_marking, dk_final_marking, rg_transition_matrix,
            diffuser, denoiser, final_res)


def evaluate_dataset_new(denoiser, diffuser, cfg, loader):
    pad_token = cfg.num_classes - 1
    denoised = []
    gt = []
    for x, y in loader:
        x = x.permute(0, 2, 1).to(cfg.device).float()
        y = y.permute(0, 2, 1).to(cfg.device).float()
        x_hat, matrix_hat, loss, seq_loss, mat_loss = \
            diffuser.sample_with_matrix(denoiser, y.shape[0], cfg.num_classes, denoiser.max_input_dim,
                                        rg_transition_matrix.shape[-1], rg_transition_matrix, x, y,
                                        cfg.predict_on)
        denoised.append(x_hat)
        gt.append(x)
    denoised = torch.cat(denoised, dim=0)
    gt = torch.cat(gt, dim=0)

    x_list = []
    y_list = []
    for x_0, y in zip(gt, denoised):
        x_tokens = torch.argmax(x_0, dim=0)
        y_tokens = torch.argmax(torch.softmax(y, dim=0).transpose(0, 1), dim=1)
        x_list.append(np.array(x_tokens[x_tokens != pad_token].cpu()))
        y_list.append(np.array(y_tokens[x_tokens != pad_token].cpu()))

    acc = np.mean([accuracy_score(x, y) for x, y in zip(x_list, y_list)])
    rec = np.mean([recall_score(x, y, average='macro', zero_division=0) for x, y in zip(x_list, y_list)])
    pre = np.mean([precision_score(x, y, average='macro', zero_division=0) for x, y in zip(x_list, y_list)])

    return gt, denoised, acc, pre, rec

In [3]:
data_path = r"../data/pickles/50_salads_unified.pkl"
with open(data_path, "rb") as f:
    data = pkl.load(f)

target, source = data['target'], data['stochastic']

In [4]:
np.mean([metrics.accuracy_score(np.argmax(t, axis=1), np.argmax(s, axis=1)) for t, s in zip(target, source)])

0.8259146587893127

In [5]:
average_entropy_data(source)

0.24932393

In [8]:
noise_argmax_accuracies = [np.mean([metrics.accuracy_score(np.argmax(t, axis=1), np.argmax(s, axis=1)) for t, s in zip(target, stochastic_ds)]) for stochastic_ds in noisy_datasets]

In [9]:
noise_entropies = [average_entropy_data(stochastic_ds) for stochastic_ds in noisy_datasets]

In [10]:
target_dir = r"D:\Projects\trace-denoise\final_runs_extended\50_salads_free"
cfg = load_experiment_config(target_dir)
cfg.device = "cuda:0"
train_dataset, test_dataset, dk_process_model, dk_init_marking, dk_final_marking, rg_transition_matrix, diffuser, denoiser, final_res = load_experiment_data_and_model(target_dir, cfg)

In [11]:
accumulators = []
results_list = []
for stochastic_ds in tqdm(noisy_datasets):
    noisy_dataset = SaladsDataset(target, stochastic_ds)
    _, noisy_test_dataset = train_test_split(noisy_dataset, train_size=cfg.train_percent, random_state=cfg.seed)
    noisy_test_loader = DataLoader(noisy_test_dataset, batch_size=cfg.batch_size, shuffle=False, num_workers=cfg.num_workers)
    accumulator, results = evaluate_dataset(denoiser, diffuser, cfg, noisy_test_loader)
    accumulators.append(accumulator)
    results_list.append(results)

  0%|          | 0/13 [00:00<?, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

...

0it [00:00, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

In [12]:
accs, pre, rec = [], [], []
pad_token = 19
for accumulator in accumulators:
    i_acc, i_pre, i_rec = [], [], []
    for x, y, x_hat in zip(accumulator['x'], accumulator['y'], accumulator['x_hat']):
        for tdk, tsk, that in zip(x, y, x_hat):
            dk = torch.argmax(tdk, dim=0).cpu().numpy()
            hat = torch.argmax(torch.softmax(that, dim=1), dim=1).cpu().numpy()
            i_acc.append(metrics.accuracy_score(dk[dk != pad_token], hat[dk != pad_token]))
            i_pre.append(metrics.precision_score(dk[dk != pad_token], hat[dk != pad_token], average='macro', zero_division=0))
            i_rec.append(metrics.recall_score(dk[dk != pad_token], hat[dk != pad_token], average='macro', zero_division=0))
    accs.append(np.mean(i_acc))
    pre.append(np.mean(i_pre))
    rec.append(np.mean(i_rec))

In [16]:
evals_new = []
accs_new = []
pre_new = []
for stochastic_ds in tqdm(noisy_datasets):
    noisy_dataset = SaladsDataset(target, stochastic_ds)
    _, noisy_test_dataset = train_test_split(noisy_dataset, train_size=cfg.train_percent, random_state=cfg.seed)
    noisy_test_loader = DataLoader(noisy_test_dataset, batch_size=cfg.batch_size, shuffle=False, num_workers=cfg.num_workers)
    gt, denoised, acc, pre, rec = evaluate_dataset_new(denoiser, diffuser, cfg, noisy_test_loader)
    evals_new.append((gt, denoised, acc, pre, rec))
    accs_new.append(acc)
    pre_new.append(pre)

  0%|          | 0/13 [00:00<?, ?it/s]

In [8]:
clean_levels = np.arange(0.9, 0.6, -0.025)
noise_levels = np.arange(0.2, 0.525, 0.025)
global_levels = np.arange(0.2, 0.33, 0.01)

In [16]:
from pathlib import Path

noisy_datasets_set = set()

for exp in tqdm(os.listdir(r"D:\Projects\trace-denoise\final_runs_extended")):
    target_dir = os.path.join(r"D:\Projects\trace-denoise\final_runs_extended", exp)
    cfg = load_experiment_config(target_dir)
    cfg.device = "cuda:0"
    if cfg.data_path not in noisy_datasets_set:
        with open(cfg.data_path, "rb") as f:
            data = pkl.load(f)
        target, source = data['target'], data['stochastic']
        noisy_datasets = [[np.array(generate_locked_bursty_trace(t, low=0.05, high=3, middle=0.5,
                                                         noise_level_clean=lv_global,
                                                         noise_level_noisy=lv_global,
                                                         p_c2n=lv_noise,
                                                         p_n2c=lv_clean)) for t in target] for lv_noise, lv_clean, lv_global in zip(noise_levels, clean_levels, global_levels)]
        noisy_datasets_set.add(cfg.data_path)
        with open(f"{Path(cfg.data_path).stem}_noisy.pkl", "wb") as f:
            pkl.dump(noisy_datasets, f)

  0%|          | 0/15 [00:00<?, ?it/s]

In [ ]:
from collections import defaultdict

all_datasets_results = defaultdict(dict)
for exp in tqdm(os.listdir(r"D:\Projects\trace-denoise\final_runs_extended")[10:]):
    target_dir = os.path.join(r"D:\Projects\trace-denoise\final_runs_extended", exp)
    cfg = load_experiment_config(target_dir)
    cfg.device = "cuda:0"
    train_dataset, test_dataset, dk_process_model, dk_init_marking, dk_final_marking, rg_transition_matrix, diffuser, denoiser, final_res = load_experiment_data_and_model(target_dir, cfg)
    base_test_loader = DataLoader(test_dataset, batch_size=cfg.batch_size, shuffle=False, num_workers=cfg.num_workers)
    gt, denoised, acc, pre, rec = evaluate_dataset_new(denoiser, diffuser, cfg, base_test_loader)
    all_datasets_results[exp]["base"] = (gt, denoised, acc, pre, rec)
    noisy_evals = []
    with open(f"{Path(cfg.data_path).stem}_noisy.pkl", "rb") as f:
        noisy_datasets = pkl.load(f)
    with open(cfg.data_path, "rb") as f:
        data = pkl.load(f)
    target, source = data['target'], data['stochastic']
    for noisy_ds in tqdm(noisy_datasets):
        noisy_dataset = SaladsDataset(target, noisy_ds)
        _, noisy_test_dataset = train_test_split(noisy_dataset, train_size=cfg.train_percent, random_state=cfg.seed)
        noisy_test_loader = DataLoader(noisy_test_dataset, batch_size=cfg.batch_size, shuffle=False, num_workers=cfg.num_workers)
        gt, denoised, acc, pre, rec = evaluate_dataset_new(denoiser, diffuser, cfg, noisy_test_loader)
        noisy_evals.append((gt, denoised, acc, pre, rec))
    all_datasets_results[exp]["noisy"] = noisy_evals
    with open(r"all_datasets_results.pkl", "wb") as f:
        pkl.dump(all_datasets_results, f)

  0%|          | 0/15 [00:00<?, ?it/s]

  0%|          | 0/13 [00:00<?, ?it/s]

C:\Users\Maxim\anaconda3\Lib\site-packages\torch_geometric\utils\convert.py:278: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\pytorch\torch\csrc\utils\tensor_new.cpp:256.)
  data_dict[key] = torch.as_tensor(value)


  0%|          | 0/13 [00:00<?, ?it/s]

  0%|          | 0/13 [00:00<?, ?it/s]

...

  0%|          | 0/13 [00:00<?, ?it/s]

  0%|          | 0/13 [00:00<?, ?it/s]

  0%|          | 0/13 [00:00<?, ?it/s]

  0%|          | 0/13 [00:00<?, ?it/s]

  0%|          | 0/13 [00:00<?, ?it/s]

In [2]:
with open("all_datasets_results.pkl", "rb") as f:
    all_datasets_results = pkl.load(f)

In [7]:
{k: v['base'][2] for k, v in all_datasets_results.items()}

{'50_salads_free': 0.8592291944696319,
 '50_salads_gnn_atn': 0.8880994210063227,
 '50_salads_matrix': 0.8673383070942174,
 'ava_free': 0.959640197053902,
 'ava_gnn': 0.9653844066668041,
 'ava_matrix': 0.9619945467121948,
 'bpi12_free': 0.9748650708975153,
 'bpi12_gnn_atn': 0.9773906311301692,
 'bpi12_matrix': 0.9786881008230166,
 'breakfast_free': 0.9304494269260867}

In [11]:
os.listdir(r"D:\Projects\trace-denoise\final_runs_extended")[10:]

['breakfast_gnn_atn',
 'breakfast_matrix',
 'gtea_free',
 'gtea_gnn_atn',
 'gtea_matrix']

In [12]:
temp = "../data/pickles/50_salads_unified.pkl"

In [13]:
from pathlib import Path

Path(temp).stem

'50_salads_unified'